In [ ]:
!pip install optuna

In [ ]:
import optuna  
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np
import lightgbm as lgbm
from optuna.integration import LightGBMPruningCallback
from sklearn.model_selection import StratifiedKFold, KFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, roc_auc_score

import plotly.offline as py
py.init_notebook_mode(connected=True)
pd.set_option('max_colwidth', 500)
pd.set_option('max_columns', 500)
pd.set_option('max_rows', 100)

import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

import warnings
warnings.filterwarnings("ignore")


In [ ]:
train = pd.read_csv('/content/data_cleaned.csv', low_memory=True)
train.head()

,primary_identifier,device_type,device_category,gender,district_name,age_group,network_stay,average_monthly_bill_amount,dusage_sum,dusage_min,dusage_max,dusage_avg,dusage_days,dusage_stddev,vusage_onnet_sum,vusage_onnet_max,vusage_onnet_min,vusage_onnet_days,vusage_onnet_avg,vusage_onnet_stddev,vusage_offnet_sum,vusage_offnet_max,vusage_offnet_min,vusage_offnet_days,vusage_offnet_avg,vusage_offnet_stddev,number_of_fixed_bb_accounts,number_of_iptv_accounts,next_month_plan
0,6875768,0,0,0,0,0,219,761.62,6317.0,2.773958,622.685848,210.566667,30.0,197.116822,0.0,0.0,0.0,0.0,0.0,0.0,120.0,38.000000,0.0,27.0,25.074074,13.000000,0.0,1.0,2
1,6406277,0,0,0,1,0,36,2482.52,28365.0,3.079438,2569.429138,945.500000,30.0,573.299639,0.0,0.0,0.0,0.0,0.0,0.0,76.0,35.433333,0.0,15.0,5.066667,6.779644,0.0,0.0,1
2,3563570,0,0,0,2,1,186,1565.55,18581.0,48.193713,2739.515326,619.366667,30.0,619.750500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,6
3,1853666,0,0,1,3,0,24,2397.49,35000.0,22.437842,4000.000000,3035.700000,30.0,800.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,4
4,2794331,0,0,1,2,2,37,2501.75,19736.0,53.302092,1239.220170,657.866667,30.0,329.152516,0.0,0.0,0.0,0.0,0.0,0.0,120.0,38.000000,0.0,23.0,11.217391,13.000000,0.0,1.0,4


In [ ]:
train['device_type'] = train['device_type'].astype(int)
train['device_category'] = train['device_category'].astype(int)
train['gender'] = train['gender'].astype(int)
train['district_name'] = train['district_name'].astype(int)
train['age_group'] = train['age_group'].astype(int)
train['next_month_plan'] = train['next_month_plan'].astype(int)
train['vusage_offnet_days'] = train['vusage_offnet_days'].astype(int)

In [ ]:
train_selected = train.set_index('primary_identifier')
selected_features = [
                     'age_group',
                     'gender',
                     'network_stay',
                     'average_monthly_bill_amount',
                     'dusage_sum',
                     'dusage_max',
                     'dusage_stddev',
                     'vusage_offnet_max',
                     'vusage_offnet_days',
                     'vusage_offnet_stddev',
                     'next_month_plan']

In [ ]:
# train_df = pd.read_csv("/content/Train_Fix.csv",low_memory=False)
# test_df = pd.read_csv("/content/Validation_Fix.csv",low_memory=False)

In [ ]:
# train_df = train_df.set_index('primary_identifier')
# test_df = test_df.set_index('primary_identifier')
# temp = train_df[selected_features]
# temp1 = test_df[selected_features]

In [ ]:
# #defining train test data
# x_train = temp[temp.columns[:-1]]

# # from sklearn.preprocessing import MinMaxScaler
# # scaler = MinMaxScaler()
# # x_train = scaler.fit_transform(x_train)

# y_train = temp[temp.columns[-1]]
# y_train = y_train.astype('int')

# x_test = temp1[temp1.columns[:-1]]
# # scaler1 = MinMaxScaler()
# # x_test = scaler1.fit_transform(x_test)

# y_test = temp1[temp1.columns[-1]]
# y_test = y_test.astype('int')


In [ ]:
# #Scaling using the Standard Scaler
# sc_1=StandardScaler()
# sc_1.fit(df_1)
# X_1=pd.DataFrame(sc_1.fit_transform(df_1))

In [ ]:
# defining train test data
train1 = train[selected_features]
X = train1[train1.columns[:-1]]
y = train1[train1.columns[-1]]

In [ ]:
y

0        2
1        1
2        6
3        4
4        4
        ..
10332    6
10333    1
10334    6
10335    1
10336    1
Name: next_month_plan, Length: 10337, dtype: int64

In [ ]:
# from imblearn.over_sampling import SMOTE
# from collections import Counter
# from matplotlib import pyplot

# #X, y = train.iloc[:, :-1], train.iloc[:, -1]


# # transform the dataset
# oversample = SMOTE()
# X, y = oversample.fit_resample(X, y)
# # summarize distribution
# counter = Counter(y)
# for k,v in counter.items():
# 	per = v / len(y) * 100
# 	print('Class=%d, n=%d (%.3f%%)' % (k, v, per))
# # plot the distribution
# pyplot.bar(counter.keys(), counter.values())
# pyplot.show()

In [ ]:
def objective(trial, X, y):
    param_grid = {
        # "device_type": trial.suggest_categorical("device_type", ['gpu']),
        "n_estimators": trial.suggest_categorical("n_estimators", [10000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        "max_bin": trial.suggest_int("max_bin", 200, 300),
        "max_bin_by_feature": trial.suggest_int("max_bin_by_feature", 3, 15),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.95, step=0.1
        ),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.2, 0.95, step=0.1
        ),
    }

    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1121218)

    cv_scores = np.empty(10)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
         X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
         # X_train, X_test = X[train_idx], X[test_idx]
         y_train, y_test = y[train_idx], y[test_idx]

         model = lgbm.LGBMClassifier(
                                    objective="multiclass", 
                                    num_class = 8,
                                    categorical_feature =(0,1),  
                                    **param_grid)
         model.fit(
            X_train,
            y_train,
            eval_set=[(X_test, y_test)],
            eval_metric="multi_logloss",
            early_stopping_rounds=100,
            callbacks=[
                LightGBMPruningCallback(trial, "multi_logloss")
            ],  # Add a pruning callback
        )
         #preds = model.predict_proba(X_test)
         preds = model.predict_proba(X_test)
         cv_scores[idx] = log_loss(y_test, preds)

    return np.mean(cv_scores)


In [ ]:
study = optuna.create_study(direction="minimize", study_name="LGBM Classifier")
func = lambda trial: objective(trial, X, y)
study.optimize(func, n_trials=20)

[I 2021-09-17 21:04:09,271] A new study created in memory with name: LGBM Classifier


[1]	valid_0's multi_logloss: 2.01736	valid_0's multi_logloss: 2.01736
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's multi_logloss: 2.01736	valid_0's multi_logloss: 2.01736
[3]	valid_0's multi_logloss: 2.01736	valid_0's multi_logloss: 2.01736
[4]	valid_0's multi_logloss: 2.01736	valid_0's multi_logloss: 2.01736
[5]	valid_0's multi_logloss: 2.01736	valid_0's multi_logloss: 2.01736
[6]	valid_0's multi_logloss: 2.01736	valid_0's multi_logloss: 2.01736
[7]	valid_0's multi_logloss: 2.01736	valid_0's multi_logloss: 2.01736
[8]	valid_0's multi_logloss: 2.01736	valid_0's multi_logloss: 2.01736
[9]	valid_0's multi_logloss: 2.01736	valid_0's multi_logloss: 2.01736
[10]	valid_0's multi_logloss: 2.01736	valid_0's multi_logloss: 2.01736
[11]	valid_0's multi_logloss: 2.01736	valid_0's multi_logloss: 2.01736
[12]	valid_0's multi_logloss: 2.01736	valid_0's multi_logloss: 2.01736
[13]	valid_0's multi_logloss: 2.01736	valid_0's multi_logloss: 2.01736
[14]	valid_0's multi_lo

[I 2021-09-17 21:04:12,701] Trial 0 finished with value: 1.8124288238919681 and parameters: {'n_estimators': 10000, 'learning_rate': 0.1628155315754351, 'num_leaves': 1040, 'max_depth': 7, 'min_data_in_leaf': 6300, 'max_bin': 215, 'max_bin_by_feature': 10, 'lambda_l1': 50, 'lambda_l2': 85, 'min_gain_to_split': 11.668195211862164, 'bagging_fraction': 0.30000000000000004, 'bagging_freq': 1, 'feature_fraction': 0.4}. Best is trial 0 with value: 1.8124288238919681.


[88]	valid_0's multi_logloss: 2.01684	valid_0's multi_logloss: 2.01684
[89]	valid_0's multi_logloss: 2.01684	valid_0's multi_logloss: 2.01684
[90]	valid_0's multi_logloss: 2.01684	valid_0's multi_logloss: 2.01684
[91]	valid_0's multi_logloss: 2.01684	valid_0's multi_logloss: 2.01684
[92]	valid_0's multi_logloss: 2.01684	valid_0's multi_logloss: 2.01684
[93]	valid_0's multi_logloss: 2.01684	valid_0's multi_logloss: 2.01684
[94]	valid_0's multi_logloss: 2.01684	valid_0's multi_logloss: 2.01684
[95]	valid_0's multi_logloss: 2.01684	valid_0's multi_logloss: 2.01684
[96]	valid_0's multi_logloss: 2.01684	valid_0's multi_logloss: 2.01684
[97]	valid_0's multi_logloss: 2.01684	valid_0's multi_logloss: 2.01684
[98]	valid_0's multi_logloss: 2.01684	valid_0's multi_logloss: 2.01684
[99]	valid_0's multi_logloss: 2.01684	valid_0's multi_logloss: 2.01684
[100]	valid_0's multi_logloss: 2.01684	valid_0's multi_logloss: 2.01684
[101]	valid_0's multi_logloss: 2.01684	valid_0's multi_logloss: 2.01684
Earl

[I 2021-09-17 21:04:16,047] Trial 1 finished with value: 1.8124288238919681 and parameters: {'n_estimators': 10000, 'learning_rate': 0.097780398461958, 'num_leaves': 360, 'max_depth': 12, 'min_data_in_leaf': 3000, 'max_bin': 246, 'max_bin_by_feature': 11, 'lambda_l1': 15, 'lambda_l2': 90, 'min_gain_to_split': 14.101338946419485, 'bagging_fraction': 0.30000000000000004, 'bagging_freq': 1, 'feature_fraction': 0.8}. Best is trial 0 with value: 1.8124288238919681.


...

[I 2021-09-17 21:06:13,870] Trial 17 finished with value: 1.8124288238919681 and parameters: {'n_estimators': 10000, 'learning_rate': 0.25788150340163984, 'num_leaves': 2600, 'max_depth': 6, 'min_data_in_leaf': 4300, 'max_bin': 285, 'max_bin_by_feature': 13, 'lambda_l1': 30, 'lambda_l2': 80, 'min_gain_to_split': 2.0961416009468636, 'bagging_fraction': 0.8, 'bagging_freq': 1, 'feature_fraction': 0.8}. Best is trial 12 with value: 1.3919312821099337.


[57]	valid_0's multi_logloss: 2.01684	valid_0's multi_logloss: 2.01684
[58]	valid_0's multi_logloss: 2.01684	valid_0's multi_logloss: 2.01684
[59]	valid_0's multi_logloss: 2.01684	valid_0's multi_logloss: 2.01684
[60]	valid_0's multi_logloss: 2.01684	valid_0's multi_logloss: 2.01684
[61]	valid_0's multi_logloss: 2.01684	valid_0's multi_logloss: 2.01684
[62]	valid_0's multi_logloss: 2.01684	valid_0's multi_logloss: 2.01684
[63]	valid_0's multi_logloss: 2.01684	valid_0's multi_logloss: 2.01684
[64]	valid_0's multi_logloss: 2.01684	valid_0's multi_logloss: 2.01684
[65]	valid_0's multi_logloss: 2.01684	valid_0's multi_logloss: 2.01684
[66]	valid_0's multi_logloss: 2.01684	valid_0's multi_logloss: 2.01684
[67]	valid_0's multi_logloss: 2.01684	valid_0's multi_logloss: 2.01684
[68]	valid_0's multi_logloss: 2.01684	valid_0's multi_logloss: 2.01684
[69]	valid_0's multi_logloss: 2.01684	valid_0's multi_logloss: 2.01684
[70]	valid_0's multi_logloss: 2.01684	valid_0's multi_logloss: 2.01684
[71]	v

[I 2021-09-17 21:06:25,886] Trial 18 finished with value: 1.2781207345554564 and parameters: {'n_estimators': 10000, 'learning_rate': 0.1966968535498656, 'num_leaves': 1700, 'max_depth': 3, 'min_data_in_leaf': 2100, 'max_bin': 259, 'max_bin_by_feature': 14, 'lambda_l1': 0, 'lambda_l2': 90, 'min_gain_to_split': 4.616796565095227, 'bagging_fraction': 0.9, 'bagging_freq': 1, 'feature_fraction': 0.7}. Best is trial 18 with value: 1.2781207345554564.


[162]	valid_0's multi_logloss: 1.28626	valid_0's multi_logloss: 1.28626
[163]	valid_0's multi_logloss: 1.28626	valid_0's multi_logloss: 1.28626
[164]	valid_0's multi_logloss: 1.28626	valid_0's multi_logloss: 1.28626
[165]	valid_0's multi_logloss: 1.28626	valid_0's multi_logloss: 1.28626
[166]	valid_0's multi_logloss: 1.28626	valid_0's multi_logloss: 1.28626
[167]	valid_0's multi_logloss: 1.28626	valid_0's multi_logloss: 1.28626
[168]	valid_0's multi_logloss: 1.28626	valid_0's multi_logloss: 1.28626
[169]	valid_0's multi_logloss: 1.28626	valid_0's multi_logloss: 1.28626
[170]	valid_0's multi_logloss: 1.28626	valid_0's multi_logloss: 1.28626
[171]	valid_0's multi_logloss: 1.28626	valid_0's multi_logloss: 1.28626
[172]	valid_0's multi_logloss: 1.28626	valid_0's multi_logloss: 1.28626
[173]	valid_0's multi_logloss: 1.28626	valid_0's multi_logloss: 1.28626
[174]	valid_0's multi_logloss: 1.28626	valid_0's multi_logloss: 1.28626
[175]	valid_0's multi_logloss: 1.28626	valid_0's multi_logloss: 

[I 2021-09-17 21:06:44,207] Trial 19 finished with value: 1.2637334731546233 and parameters: {'n_estimators': 10000, 'learning_rate': 0.13542048929041509, 'num_leaves': 1280, 'max_depth': 4, 'min_data_in_leaf': 1700, 'max_bin': 228, 'max_bin_by_feature': 9, 'lambda_l1': 0, 'lambda_l2': 50, 'min_gain_to_split': 2.436953851241351, 'bagging_fraction': 0.7, 'bagging_freq': 1, 'feature_fraction': 0.7}. Best is trial 19 with value: 1.2637334731546233.


In [ ]:
print(f"\tBest value (rmse): {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

	Best value (rmse): 1.26373
	Best params:
		n_estimators: 10000
		learning_rate: 0.13542048929041509
		num_leaves: 1280
		max_depth: 4
		min_data_in_leaf: 1700
		max_bin: 228
		max_bin_by_feature: 9
		lambda_l1: 0
		lambda_l2: 50
		min_gain_to_split: 2.436953851241351
		bagging_fraction: 0.7
		bagging_freq: 1
		feature_fraction: 0.7
